# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps.datasets
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Loan CSV file generated from WeatherPy Folder
output_data_to_load = "../output_data/cities.csv"
city_df = pd.read_csv(output_data_to_load)
city_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kruisfontein,-34.0033,24.7314,78.89,44,0,10.47,ZA,1644133061
1,Coyhaique,-45.5752,-72.0662,49.95,87,0,0.00,CL,1644133062
2,Nouakchott,18.0858,-15.9785,62.53,40,0,11.05,MR,1644133062
3,Arraial do Cabo,-22.9661,-42.0278,75.15,90,45,22.03,BR,1644133063
4,Verkhoyansk,67.5447,133.3850,-40.83,100,92,0.92,RU,1644133063
...,...,...,...,...,...,...,...,...,...
585,Constitución,-35.3333,-72.4167,61.25,67,0,4.88,CL,1644133424
586,Quelimane,-17.8786,36.8883,84.15,74,40,12.66,MZ,1644133425
587,Goundi,9.3627,17.3660,81.32,14,13,7.96,TD,1644133425
588,Roald,62.5833,6.1333,36.34,80,40,16.11,NO,1644133426


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps

gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity =city_df["Humidity"]


In [4]:

# Plot Heatmap   

locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 3


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
clean_df = city_df[(city_df["Max Temp"] >= 75) & (city_df["Max Temp"] <= 90)]
clean_df = clean_df[clean_df["Wind Speed"] <= 10]
clean_df = clean_df[clean_df["Cloudiness"] <= 10]
clean_df = clean_df[clean_df["Humidity"] <= 70]
hotel_df = clean_df
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
104,Kavaratti,10.5669,72.6420,78.37,62,7,6.51,IN,1644133108
140,Cape Town,-33.9258,18.4232,84.11,46,0,5.75,ZA,1644132901
305,Salalah,17.0151,54.0924,80.69,61,0,4.61,OM,1644133278
324,Athani,16.7333,75.0667,85.23,24,10,7.81,IN,1644133288
405,Plettenberg Bay,-34.0527,23.3716,79.03,55,0,8.21,ZA,1644133328
472,Parakou,9.3372,2.6303,75.74,17,0,7.92,BJ,1644133361
516,Surgāna,20.5667,73.6167,84.85,16,1,6.42,IN,1644133383
559,Porbandar,21.6422,69.6093,76.98,44,0,5.75,IN,1644133409
584,Alice Springs,-23.7000,133.8833,85.55,25,0,5.75,AU,1644133396


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
104,Kavaratti,10.5669,72.6420,78.37,62,7,6.51,IN,1644133108,
140,Cape Town,-33.9258,18.4232,84.11,46,0,5.75,ZA,1644132901,
305,Salalah,17.0151,54.0924,80.69,61,0,4.61,OM,1644133278,
324,Athani,16.7333,75.0667,85.23,24,10,7.81,IN,1644133288,
405,Plettenberg Bay,-34.0527,23.3716,79.03,55,0,8.21,ZA,1644133328,
472,Parakou,9.3372,2.6303,75.74,17,0,7.92,BJ,1644133361,
516,Surgāna,20.5667,73.6167,84.85,16,1,6.42,IN,1644133383,
559,Porbandar,21.6422,69.6093,76.98,44,0,5.75,IN,1644133409,
584,Alice Springs,-23.7000,133.8833,85.55,25,0,5.75,AU,1644133396,


In [7]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df        

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
104,Kavaratti,10.5669,72.6420,78.37,62,7,6.51,IN,1644133108,Hotels in Lakshadweep Islands
140,Cape Town,-33.9258,18.4232,84.11,46,0,5.75,ZA,1644132901,"Radisson Blu Hotel Waterfront, Cape Town"
305,Salalah,17.0151,54.0924,80.69,61,0,4.61,OM,1644133278,Salalah Gardens Hotel
324,Athani,16.7333,75.0667,85.23,24,10,7.81,IN,1644133288,Pattan's Hotel Prathamesh Arcade
405,Plettenberg Bay,-34.0527,23.3716,79.03,55,0,8.21,ZA,1644133328,Bayview Hotel
472,Parakou,9.3372,2.6303,75.74,17,0,7.92,BJ,1644133361,Kobourou City Hotel
516,Surgāna,20.5667,73.6167,84.85,16,1,6.42,IN,1644133383,NaN
559,Porbandar,21.6422,69.6093,76.98,44,0,5.75,IN,1644133409,Lords Inn Porbandar
584,Alice Springs,-23.7000,133.8833,85.55,25,0,5.75,AU,1644133396,"Crowne Plaza Alice Springs Lasseters, an IHG H..."


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))